In [1]:
import pandas as pd
import numpy as np
import altair

In [2]:
# load the starts data
starts_df = pd.read_csv('data/skillshare_2022_starts.csv', index_col=0)
starts_df.head()

/tmp/ipykernel_887117/494318024.py:2: DtypeWarning: Columns (24,53) have mixed types. Specify dtype option on import or set low_memory=False.
  starts_df = pd.read_csv('data/skillshare_2022_starts.csv', index_col=0)


,id,user_uid,is_active,create_time,update_time,realized_end_time,expiration_time,first_payment_time,last_payment_time,next_billing_time,...,is_direct_to_paid,trial_length_offer,had_trial,sub_utm_source,sub_utm_campaign,sub_utm_medium,sub_utm_term,sub_utm_channel,referral_source,eligible_trial_number
0,8794519,14996631.0,False,2022-01-01 00:12:27,2022-01-01 00:12:27,2022-06-23 12:56:46,NaN,NaN,NaN,NaN,...,False,None,False,ShortUrl,teacher-referral,teacher-referral,NaN,Teacher Referral,https://www.linkedin.com/,NaN
1,8794529,23693211.0,False,2022-01-01 00:15:05,2022-01-08 18:32:43,2022-01-08 18:32:43,2022-01-08 18:32:43,2022-01-08 00:17:31,2022-01-08 00:17:31,NaN,...,False,One Week,True,NaN,NaN,NaN,NaN,Organic Search,https://www.google.com/,1.0
2,8794531,23693219.0,True,2022-01-01 00:17:15,2022-01-01 00:17:15,2022-06-23 12:56:46,2023-01-31 10:00:00,2022-01-31 11:27:12,2022-01-31 11:27:12,NaN,...,False,One Month,True,Google,Brand_EN-UK_IE,paidsearch,how much is skillshare,Paid Search (Brand),NaN,1.0
3,8794558,23693241.0,False,2022-01-01 00:25:49,2022-02-01 03:31:19,2022-02-01 03:31:19,2022-02-01 03:31:19,NaN,NaN,NaN,...,False,One Month,True,youtube,2021-04-01-peter-mckinnon,paid-influencer-video,NaN,Youtube Influencer,https://www.youtube.com/,1.0
4,8794560,11790171.0,True,2022-01-01 00:26:54,2022-01-01 00:26:59,2022-06-23 12:56:46,NaN,2022-01-01 00:26:56,2022-01-01 00:26:56,2023-01-01 00:26:55,...,False,None,False,Google,Brand_EN-US,paidsearch,skillshare,Paid Search (Brand),https://www.google.com/,NaN


In [3]:
# clean the starts data.
# Limit to those with a yearly subscription
starts_df = starts_df[starts_df['plan_length'] == 12]
# Limit to those not provided free subscriptions already
starts_df = starts_df[starts_df['is_team'] == False]
starts_df = starts_df[starts_df['is_scholarship'] == False]
starts_df = starts_df[starts_df['is_direct_to_paid'] == False]
starts_df = starts_df[starts_df['had_trial'] == True]
starts_df = starts_df[starts_df['trial_length_offer'].isin(['One Week', 'One Month'])]

In [4]:
# all of these columns have the chance to be valuable to us.
valuable_cols = ['user_uid', 'is_active', 'create_time',
       'first_payment_time', 'last_payment_time', 'next_billing_time',
       'last_payment_attempt', 'last_failed_payment_attempt',
       'user_cancellation_time', 'cancellation_time', 'refund_time', 'coupon_id',
       'coupon_trial_length', 'payment_provider', 'payment_ux', 'is_paused',
       'pause_time', 'is_refunded', 'is_resume', 'paid_through', 'is_lapsed', 'is_pending', 'is_cancelled',
       'has_paid', 'is_paid', 'trial_end', 'category',
       'num_payments', 'num_successful_payments',
       'first_payment_currency_code', 'subscription_number',
       'paid_subscription_number', 'eligible_subscription_number',
       'is_p1_refunded', 'trial_extension_time',
       'original_create_time', 'original_trial_end', 'extended_trial_end',
       'was_trial_extended', 'is_trial_extension', 'is_split_trial',
       'is_paid_reactivation', 'trial_length_days', 'trial_length_offer', 'sub_utm_source', 'sub_utm_campaign',
       'sub_utm_medium', 'sub_utm_term', 'sub_utm_channel', 'referral_source',
       'eligible_trial_number']

starts_big_clean_df = starts_df[valuable_cols]

In [5]:
# these are the columns I bet will have some value to prediction.
prediction_cols = ['user_uid', 'create_time', 'first_payment_time', 'last_payment_attempt',
                   'last_failed_payment_attempt', 'user_cancellation_time', 'cancellation_time', 
                   'refund_time', 'coupon_id', 'coupon_trial_length', 'payment_provider', 'payment_ux', 
                   'is_refunded', 'has_paid', 'trial_end', 'first_payment_currency_code', 'original_trial_end', 
                   'extended_trial_end', 'was_trial_extended', 'is_trial_extension', 'is_split_trial', 
                   'trial_length_days', 'trial_length_offer', 'sub_utm_source', 'sub_utm_campaign', 
                   'sub_utm_medium', 'sub_utm_term', 'sub_utm_channel', 'referral_source', 'eligible_trial_number']

starts_clean_df = starts_big_clean_df[prediction_cols]
starts_clean_df.head()

,user_uid,create_time,first_payment_time,last_payment_attempt,last_failed_payment_attempt,user_cancellation_time,cancellation_time,refund_time,coupon_id,coupon_trial_length,...,is_split_trial,trial_length_days,trial_length_offer,sub_utm_source,sub_utm_campaign,sub_utm_medium,sub_utm_term,sub_utm_channel,referral_source,eligible_trial_number
1,23693211.0,2022-01-01 00:15:05,2022-01-08 00:17:31,2022-01-08 00:17:31,NaN,2022-01-08 05:35:07,2022-01-08 05:35:07,2022-01-08 05:35:07,NaN,NaN,...,False,7.0,One Week,NaN,NaN,NaN,NaN,Organic Search,https://www.google.com/,1.0
2,23693219.0,2022-01-01 00:17:15,2022-01-31 11:27:12,2022-01-31 11:27:12,NaN,2022-01-31 15:29:26,2022-01-31 15:29:26,NaN,102027.0,1.0,...,False,31.0,One Month,Google,Brand_EN-UK_IE,paidsearch,how much is skillshare,Paid Search (Brand),NaN,1.0
3,23693241.0,2022-01-01 00:25:49,NaN,NaN,NaN,2022-01-21 15:08:34,2022-01-21 15:08:34,NaN,102121.0,1.0,...,False,31.0,One Month,youtube,2021-04-01-peter-mckinnon,paid-influencer-video,NaN,Youtube Influencer,https://www.youtube.com/,1.0
5,12529812.0,2022-01-01 00:40:01,NaN,NaN,NaN,NaN,NaN,NaN,6734.0,1.0,...,False,31.0,One Month,ShortUrl,affiliate-referral-1941759612,affiliate-referral,NaN,Affiliate Referral,https://justcreative.com/,NaN
6,23673916.0,2022-01-01 00:40:30,NaN,NaN,NaN,2022-01-01 01:19:39,2022-01-01 01:19:39,NaN,NaN,NaN,...,False,7.0,One Week,NaN,NaN,NaN,NaN,Organic Search,https://www.google.com/,1.0


In [6]:
# make a day version of the trial start and end dates.
starts_clean_df['trial_end_day'] = pd.to_datetime(starts_clean_df.original_trial_end).dt.date
starts_clean_df['trial_start_day'] = pd.to_datetime(starts_clean_df.create_time).dt.date

# for some reason user_uid is a float only on this table. Change it to an int.
starts_clean_df['user_uid'] = starts_clean_df['user_uid'].astype(int)

/tmp/ipykernel_887117/811030169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starts_clean_df['trial_end_day'] = pd.to_datetime(starts_clean_df.original_trial_end).dt.date
/tmp/ipykernel_887117/811030169.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starts_clean_df['trial_start_day'] = pd.to_datetime(starts_clean_df.create_time).dt.date
/tmp/ipykernel_887117/811030169.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [7]:
converted = starts_clean_df.first_payment_time.notnull().sum()
didnt_convert = starts_clean_df.first_payment_time.isna().sum()

print(f"Conversion rate: {converted / (converted + didnt_convert):.2f}")

converted_counts = pd.DataFrame(
    {
        "Converted": [converted],
        "Didn't convert": [didnt_convert]
    })\
    .T\
    .reset_index()\
    .rename(
        columns={
            0:"Users", 
            "index":"class"})\


altair.Chart(
    converted_counts,
    title="Trial Conversions")\
    .mark_bar()\
    .encode(
        x=altair.X("Users:Q"),
        y=altair.Y("class:N", title=None))

Conversion rate: 0.30


alt.Chart(...)

In [8]:
print(starts_clean_df.first_payment_time.dtype)
starts_clean_df.first_payment_time = pd.to_datetime(starts_clean_df.first_payment_time)
starts_clean_df.create_time = pd.to_datetime(starts_clean_df.create_time)
starts_clean_df['day_converted_after_trial_start'] = starts_clean_df.first_payment_time - starts_clean_df.create_time
starts_clean_df.day_converted_after_trial_start.isna().count()

/tmp/ipykernel_887117/836677339.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starts_clean_df.first_payment_time = pd.to_datetime(starts_clean_df.first_payment_time)


object


/tmp/ipykernel_887117/836677339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starts_clean_df.create_time = pd.to_datetime(starts_clean_df.create_time)
/tmp/ipykernel_887117/836677339.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starts_clean_df['day_converted_after_trial_start'] = starts_clean_df.first_payment_time - starts_clean_df.create_time


483026

In [9]:
# make a small df for merging onto the video views data.
trial_ends = starts_clean_df[['user_uid', 'trial_start_day', 'trial_end_day']]

In [10]:
# load the video views data.
vviews1_df = pd.read_csv('data/skillshare_2022_vviews_0.csv', index_col=0)
vviews1_df

,uid,class_id,lesson_id,lesson_number,is_trailer,video_duration,view_date,sum
0,1128075,121998,1533011,6,False,407.0,2022-01-07,333
1,23721429,285819,2992845,3,False,379.0,2022-01-07,215
2,23747223,151243,2482675,5,False,372.0,2022-01-07,192
3,23630537,108140,652188,19,False,203.0,2022-01-07,24
4,23693292,516912,4244299,3,False,340.0,2022-01-07,89
...,...,...,...,...,...,...,...,...
323470,22428068,560627,4548235,6,False,451.0,2022-05-28,28
323471,22428068,560627,4549913,10,False,964.0,2022-05-28,3
323472,22428068,560627,4552219,5,False,856.0,2022-05-28,11
323473,22428068,555448,4462180,0,True,122.0,2022-05-28,23


In [11]:
# TODO: is this a problem?
vviews1_df.isna().any()

uid               False
class_id          False
lesson_id         False
lesson_number     False
is_trailer        False
video_duration     True
view_date         False
sum               False
dtype: bool

In [12]:
# change str view_date into dt.
vviews1_df['view_date_dt'] = pd.to_datetime(vviews1_df.view_date).dt.date

In [13]:
# merge the trial start and end dates onto video views.
vviews1_df = vviews1_df.merge(trial_ends, how='left', left_on='uid', right_on='user_uid')

In [14]:
# filter to views that happened during the trial
trial_vviews1_df = vviews1_df[vviews1_df['view_date_dt'] < vviews1_df['trial_end_day']]

In [15]:
# add a column that tells what day of the trial the view occured, 
# thus normalizing everyone to the same "start date" of "1"
trial_vviews1_df['day_of_trial'] = (trial_vviews1_df['view_date_dt'] - trial_vviews1_df['trial_start_day']).dt.days + 1

/tmp/ipykernel_887117/2034521335.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_vviews1_df['day_of_trial'] = (trial_vviews1_df['view_date_dt'] - trial_vviews1_df['trial_start_day']).dt.days + 1


In [16]:
# some folks have had multiple trials, so they can have video views from dates prior to the trial dates here.
# preferrably we would split that into two different trials and analyze
# this isn't perfect, but for now, I'm just filtering out the previous trials and only considering
# views from the trial that fit the date range.
trial_vviews1_df = trial_vviews1_df[trial_vviews1_df['day_of_trial'] > 0]
trial_vviews1_df.head()

,uid,class_id,lesson_id,lesson_number,is_trailer,video_duration,view_date,sum,view_date_dt,user_uid,trial_start_day,trial_end_day,day_of_trial
0,1128075,121998,1533011,6,False,407.0,2022-01-07,333,2022-01-07,1128075.0,2022-01-04,2022-02-03,4
1,23721429,285819,2992845,3,False,379.0,2022-01-07,215,2022-01-07,23721429.0,2022-01-02,2022-02-02,6
3,23630537,108140,652188,19,False,203.0,2022-01-07,24,2022-01-07,23630537.0,2022-01-01,2022-02-01,7
5,23763607,514249,4230122,1,False,182.0,2022-01-07,168,2022-01-07,23763607.0,2022-01-04,2022-02-04,4
6,23717937,427040,3594641,2,False,521.0,2022-01-07,340,2022-01-07,23717937.0,2022-01-01,2022-02-01,7


In [17]:
# group by minutes watched by day.  Not super sophisticated.  Tons could be done here.
ttv_df = trial_vviews1_df.groupby(by=['uid', 'day_of_trial']).agg(minutes_watched=pd.NamedAgg(column='sum', aggfunc='sum')).reset_index()
ttv_df.head()

,uid,day_of_trial,minutes_watched
0,526,1,2229
1,526,2,65
2,526,4,1177
3,110634,1,840
4,110634,2,925


In [18]:
viz_df = trial_vviews1_df.groupby('day_of_trial')\
   .mean()['sum']\
   .reset_index()

In [19]:
altair.Chart(
    viz_df,
    title="Total Minutes Watched on Each Day of Trial")\
    .mark_circle()\
    .encode(
        x=altair.X("day_of_trial:O"),
        y=altair.Y("sum:Q", title="Minutes"),
    )

alt.Chart(...)

In [20]:
# pivot the data and fill nas with 0
vviews_pivot = ttv_df.pivot_table(
    index=['uid'],
    columns='day_of_trial',
    values='minutes_watched').reset_index().rename_axis(None, axis=1).reset_index(drop=True).fillna(0.0)

In [21]:
vviews_pivot

,uid,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,526,2229.0,65.0,0.0,1177.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,110634,840.0,925.0,0.0,1173.0,0.0,1197.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,119298,895.0,51.0,0.0,719.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,247585,621.0,0.0,3259.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,297525,4070.0,0.0,0.0,2748.0,3077.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,864.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5142,23789787,2124.0,2216.0,0.0,1185.0,0.0,0.0,2881.0,565.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5143,23789860,4353.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1062.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5144,23789963,2742.0,1481.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5145,23790057,1469.0,0.0,2236.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
